In [3]:
from rchess import Board
import numpy as np

In [5]:
from model import load_as_trt_model
from utils import FakeTRTFunc
import numpy as np
from configs import selfplayConfig
from mcts import MCTS, debug_search

mctsSearch = MCTS(selfplayConfig)
use_fake = True
if use_fake:
    trt_func = FakeTRTFunc()
else:
    trt_func, _ = load_as_trt_model()


In [7]:
board = Board()
board.push_uci("e2e4")
board.push_uci("e7e5")
board.push_uci("f1c4")
board.push_uci("b8c6")
board.push_uci("d1h5")
#board.push_uci("g8f6")
root = None
move, root, child_visits = mctsSearch.find_best_move(board, root, trt_func, 10000, 0.0, True)
debug_search(board, root)


  +---+---+---+---+---+---+---+---+
  | r |   | b | q | k | b | n | r | 8
  +---+---+---+---+---+---+---+---+
  | p | p | p | p |   | p | p | p | 7
  +---+---+---+---+---+---+---+---+
  |   |   | n |   |   |   |   |   | 6
  +---+---+---+---+---+---+---+---+
  |   |   |   |   | p |   |   | Q | 5
  +---+---+---+---+---+---+---+---+
  |   |   | B |   | P |   |   |   | 4
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 3
  +---+---+---+---+---+---+---+---+
  | P | P | P | P |   | P | P | P | 2
  +---+---+---+---+---+---+---+---+
  | R | N | B |   | K |   | N | R | 1
  +---+---+---+---+---+---+---+---+
    a   b   c   d   e   f   g   h

Fen: r1bqkbnr/pppp1ppp/2n5/4p2Q/2B1P3/8/PPPP1PPP/RNB1K1NR b KQkq - 3 3
Key: 73837f494a5e1e15
Elapsed time: 2.54s
Visits: 10000
Eval:  0.507609
Move             Visits     Policy         Avg. value       UCB              Q+U                Raw NN Value
------------------------------------------------------------------------------------

In [10]:
import timeit
num_sims = 800
#Average time 0.06498299805999977
def find_one():
    board = Board()
    root = None
    move, root, child_visits = mctsSearch.find_best_move(board, root, trt_func, num_sims, False)

# Time with timeit
time = timeit.timeit(find_one, number=50)
print("Average time", time / 50)

Average time 0.054438061000000745


In [11]:
import pstats, cProfile
#import mcts
board = Board()
cProfile.runctx("find_best_move(board, None, trt_func, 15000, True)", globals(), locals(), "Profile.prof")

s = pstats.Stats("Profile.prof")
s.strip_dirs().sort_stats("time").print_stats()

Thu Jan  9 20:52:23 2025    Profile.prof

         3705837 function calls (3698318 primitive calls) in 2.034 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      939    0.329    0.000    0.329    0.000 {built-in method tensorflow.python._pywrap_tfe.TFE_Py_Execute}
    71558    0.261    0.000    0.324    0.000 c.pyx:225(select_child)
      2/1    0.211    0.106    1.984    1.984 c.pyx:64(find_best_move)
      939    0.170    0.000    0.171    0.000 constant_op.py:75(convert_to_eager_tensor)
    15000    0.139    0.000    0.349    0.000 c.pyx:261(evaluate_node)
      939    0.122    0.000    1.014    0.001 c.pyx:210(make_predictions)
   356428    0.112    0.000    0.209    0.000 c.pyx:44(add_child)
   356429    0.079    0.000    0.093    0.000 c.pyx:24(__cinit__)
  1564821    0.059    0.000    0.059    0.000 c.pyx:250(UCB)
        1    0.050    0.050    2.034    2.034 <string>:1(<module>)
    14999    0.027    0.000    0.045

In [3]:
from rchess import Cache
import time

In [5]:
cache = Cache()
num_items = 100

start_time = time.time()
for i in range(num_items):
    cache.set(i, i)
end_time = time.time()

print(f"Time taken to insert {num_items} items: {end_time - start_time} seconds")

Time taken to insert 100 items: 0.00013494491577148438 seconds


In [43]:
import multiprocessing
from multiprocessing.managers import BaseManager
import random
import time

def insert_numbers(cache, num_items):
    for _ in range(num_items):
        num = random.randint(1, 5)
        cache.set(num, num)

def read_numbers(cache, num_items, hit_counter):
    for _ in range(num_items):
        num = random.randint(1, 10)
        if cache.get(num) is not None:
            hit_counter.value += 1

if __name__ == "__main__":
    BaseManager.register('Cache', Cache)
    manager = BaseManager()
    manager.start()
    cache = manager.Cache()
    num_items = 100
    hit_counter1 = multiprocessing.Value('i', 0)
    hit_counter2 = multiprocessing.Value('i', 0)

    p1 = multiprocessing.Process(target=insert_numbers, args=(cache, num_items))
    p2 = multiprocessing.Process(target=read_numbers, args=(cache, num_items, hit_counter1))
    p3 = multiprocessing.Process(target=read_numbers, args=(cache, num_items, hit_counter2))

    start_time = time.time()

    p1.start()
    p2.start()
    p3.start()

    p1.join()
    p2.join()
    p3.join()

    end_time = time.time()

    print(f"Time taken: {end_time - start_time} seconds")
    print(f"Cache hits by process 2: {hit_counter1.value}")
    print(f"Cache hits by process 3: {hit_counter2.value}")

Time taken: 0.0209195613861084 seconds
Cache hits by process 2: 51
Cache hits by process 3: 51


In [2]:
import numpy as np
from utils import FakeTRTFunc
from rchess import Board
import timeit
#from mcts import find_best_move, debug_search

trt_func = FakeTRTFunc()

board = Board()
board.push_uci("e2e4")
board.push_uci("e7e5")
board.push_uci("f1c4")
board.push_uci("b8c6")
board.push_uci("d1h5")
board.push_uci("g8f6")
board.push_uci("h5f7")
print(board)
""" #print(board.legal_moves_uci())
root = None
move, root, child_visits = find_best_move(board, root, trt_func, 8000, True)
debug_search(board, root) """


  +---+---+---+---+---+---+---+---+
  | r |   | b | q | k | b |   | r | 8
  +---+---+---+---+---+---+---+---+
  | p | p | p | p |   | Q | p | p | 7
  +---+---+---+---+---+---+---+---+
  |   |   | n |   |   | n |   |   | 6
  +---+---+---+---+---+---+---+---+
  |   |   |   |   | p |   |   |   | 5
  +---+---+---+---+---+---+---+---+
  |   |   | B |   | P |   |   |   | 4
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 3
  +---+---+---+---+---+---+---+---+
  | P | P | P | P |   | P | P | P | 2
  +---+---+---+---+---+---+---+---+
  | R | N | B |   | K |   | N | R | 1
  +---+---+---+---+---+---+---+---+
    a   b   c   d   e   f   g   h

Fen: r1bqkb1r/pppp1Qpp/2n2n2/4p3/2B1P3/8/PPPP1PPP/RNB1K1NR b KQkq - 0 4
Key: ce5d34984bb646a5


' #print(board.legal_moves_uci())\nroot = None\nmove, root, child_visits = find_best_move(board, root, trt_func, 8000, True)\ndebug_search(board, root) '

In [3]:
debug_search(board, root[4015])


  +---+---+---+---+---+---+---+---+
  | r |   | b | q | k | b | n | r | 8
  +---+---+---+---+---+---+---+---+
  | p | p | p | p |   | p | p | p | 7
  +---+---+---+---+---+---+---+---+
  |   |   | n |   |   |   |   |   | 6
  +---+---+---+---+---+---+---+---+
  |   |   |   |   | p |   |   | Q | 5
  +---+---+---+---+---+---+---+---+
  |   |   | B |   | P |   |   |   | 4
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 3
  +---+---+---+---+---+---+---+---+
  | P | P | P | P |   | P | P | P | 2
  +---+---+---+---+---+---+---+---+
  | R | N | B |   | K |   | N | R | 1
  +---+---+---+---+---+---+---+---+
    a   b   c   d   e   f   g   h

Fen: r1bqkbnr/pppp1ppp/2n5/4p2Q/2B1P3/8/PPPP1PPP/RNB1K1NR b KQkq - 3 3
Key: 73837f494a5e1e15
Elapsed time: 0.00s
Visits: 213
Eval:  0.583632
Move             Visits     Policy         Avg. value       UCB              Q+U                Raw NN Value
--------------------------------------------------------------------------------------